Tisya Sharma 002903942

Professor Miguel Fuentes-Cabrera

8 October, 2025

DS4400: Machine Learning and Data Mining

Midterm Take Home Coding

I declare that the following work is entirely my own, and that I did not copy or seek help from any students who
have currently or previously taken this course, nor from any online source (incl. AI) other than private messages
between myself and the professor on Piazza/via email.

-- Tisya Sharma


In [1]:
import autograd.numpy as np
from autograd import grad   # importing derivative
from autograd.numpy import log, exp # importing log function, and exponential function
from scipy.integrate import quad

Problem 1 (a)

In [2]:
def f(x):
    return log(exp(-3*x)) / log(2)

df = grad(f)

x = np.array([1.0, 2.0, 3.0])

for num in x:
    print(f" x = {num}, f'(x) = {df(num)}")

 x = 1.0, f'(x) = -4.32808512266689
 x = 2.0, f'(x) = -4.32808512266689
 x = 3.0, f'(x) = -4.32808512266689


Problem 1 (b)

In [3]:
def f(x):
    return np.sqrt(5*x - 2)

df = grad(f)

x = np.array([1.0, 2.0, 3.0])

for num in x:
    print(f"x = {num:.1f}, f'(x) = {df(num):.6f}")

x = 1.0, f'(x) = 1.443376
x = 2.0, f'(x) = 0.883883
x = 3.0, f'(x) = 0.693375


Problem 1 (c)

In [4]:
# define the integrand
def integrand(x, y):
    return np.exp(-x) + 2*x + y

# integrate inner expression
def inner_integral(y):
    result, _ = quad(integrand, 0, 1, args=(y,))
    return result

# integrate full expression
outer, _ = quad(inner_integral, 0, 1)
print("Answer:", outer)

Answer: 2.1321205588285577


Problem 1 (d)

In [5]:
Y = np.array([[0, 5, -1, 3],
              [2, 0,  1, 0],
              [0, -2, 0, 1]])

Z = np.array([[4, 1],
              [-1, 1]])

# calculate Tr(YY^T) - Tr(Z^T Z)
term1 = np.trace(np.dot(Y, Y.T))
term2 = np.trace(np.dot(Z.T, Z))
result = term1 - term2

print("Tr(YY^T) =", term1)
print("Tr(Z^T Z) =", term2)
print("Final answer:", result)

Tr(YY^T) = 45
Tr(Z^T Z) = 19
Final answer: 26


Problem 1 (e)

In [6]:
w = np.array([[1],
              [-1],
              [2]])

y = np.array([[-2],
              [1]])

A = np.array([[5, 0, 1],
              [-3, 0, 1]])

result = np.dot((np.dot(A, w) - y).T, (np.dot(A, w) - y))

print("Result =", result)

Result = [[85]]


Problem 2 (a)

In [7]:
# Data Preprocessing for Linear Regression
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Read the dataset
spotify = pd.read_csv("ds4400_spotify.csv")
display(spotify.head())

# Store song/artist names separately for later reference, then drop song name and artist names from spotify df
song_info = spotify[['song_title', 'artist_name']] if {'song_title','artist_name'}.issubset(spotify.columns) else None
spotify = spotify.drop(columns=["song_title", "artist_name"], errors="ignore")

# Separate target and features
y = spotify['danceability']
X = spotify.drop(columns=['danceability'])

# One-hot encode categorical features
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
dummy_mask = X.apply(lambda s: set(s.dropna().unique()) <= {0, 1})
dummy_cols = X.columns[dummy_mask]
X[dummy_cols] = X[dummy_cols].astype("int8")

# Normalize numeric features
# Scale only continuous numeric columns (leave dummies as 0 and 1)
cont_cols = X.columns.difference(dummy_cols)
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[cont_cols] = scaler.fit_transform(X_scaled[cont_cols])

# Build final feature map Phi with intercept
Phi = X_scaled.copy()
Phi.insert(0, 'intercept', 1.0)

# Show the feature map
display(Phi.head())
display(y.head())

,song_title,artist_name,danceability,energy,loudness,acousticness,tempo,mode
0,Hawk Em,Pop Smoke,0.680,0.607,-5.523,0.2640,141.992,Major
1,Faith,The Weeknd,0.370,0.603,-7.268,0.6120,175.962,Minor
2,Greek Tragedy - Oliver Nelson TikTok Remix,The Wombats,0.736,0.764,-6.736,0.2220,112.033,Minor
3,Homesick,Noah Kahan,0.505,0.624,-6.526,0.0253,170.076,Major
4,August,flipturn,0.450,0.602,-4.301,0.0892,172.161,Major


,intercept,energy,loudness,acousticness,tempo,mode_Minor
0,1.0,-0.041569,0.622986,-0.157528,0.537979,0
1,1.0,-0.068127,-0.039500,1.193461,1.514556,1
2,1.0,1.000835,0.162473,-0.320578,-0.323289,1
3,1.0,0.071303,0.242199,-1.084197,1.345344,0
4,1.0,-0.074767,1.086916,-0.836128,1.405284,0


0    0.680
1    0.370
2    0.736
3    0.505
4    0.450
Name: danceability, dtype: float64

Problem 2 (b)

In [8]:
# Linear regression with gradient descent

# (n, p) from part (a)
X = Phi.values.astype(float)       

# (n, 1)
y_vec = y.values.reshape(-1, 1).astype(float)   
n, p = X.shape

def mse_and_grad(w):
    r = np.dot(X, w) - y_vec
    loss = np.mean(r**2)
    grad = (2.0/n) * np.dot(X.T, r)
    return loss, grad

w = np.zeros((p, 1))
lr = 0.05
max_iter = 10000
tol = 1e-8

loss_hist = []
for i in range(max_iter):
    loss, g = mse_and_grad(w)
    loss_hist.append(loss)
    w -= lr * g
    if i > 0 and abs(loss_hist[-2] - loss_hist[-1]) < tol:
        break

w_gd = w

# Print convergence 
final_mse   = round(float(loss_hist[-1]), 6)

print("Converged in", (i + 1), "iterations\nMSE:", round(final_mse, 6))

# Final weights 
print("Final weights (gd):")
for i, col in enumerate(Phi.columns):
    val = round(float(w_gd[i, 0]), 6)
    print(col, "=", val)

# learned function string (simple building)
eq_gd = "f(x) = " + str(round(float(w_gd[0, 0]), 6))
for i, col in enumerate(Phi.columns[1:], start=1):
    eq_gd += " + " + str(round(float(w_gd[i, 0]), 6)) + " * " + col
print(eq_gd)

Converged in 261 iterations
MSE: 0.018345
Final weights (gd):
intercept = 0.563529
energy = -0.010883
loudness = -0.003718
acousticness = -0.036347
tempo = -0.087665
mode_Minor = 0.111908
f(x) = 0.563529 + -0.010883 * energy + -0.003718 * loudness + -0.036347 * acousticness + -0.087665 * tempo + 0.111908 * mode_Minor


Problem 2 (c)

In [9]:
# Linear regression via closed-form solution

# use Phi and y
X = Phi.values.astype(float)                     
y_vec = y.values.reshape(-1, 1).astype(float) 

# w = (X^T X)^(-1) X^T y
w_cf = np.dot(np.linalg.pinv(X), y_vec)

# final mse
mse_cf = float(np.mean((np.dot(X, w_cf) - y_vec) ** 2))
print("Final MSE (closed-form):", round(mse_cf, 6))

# print weights
print("final weights (closed-form):")
for i, col in enumerate(Phi.columns):
    print(col, "=", round(float(w_cf[i, 0]), 6))

# learned function
eq_cf = "f(x) = " + str(round(float(w_cf[0, 0]), 6))
for i, col in enumerate(Phi.columns[1:], start=1):
    eq_cf += " + " + str(round(float(w_cf[i, 0]), 6)) + " * " + col
print(eq_cf)

Final MSE (closed-form): 0.018344
final weights (closed-form):
intercept = 0.563945
energy = -0.010939
loudness = -0.002962
acousticness = -0.035658
tempo = -0.087951
mode_Minor = 0.110907
f(x) = 0.563945 + -0.010939 * energy + -0.002962 * loudness + -0.035658 * acousticness + -0.087951 * tempo + 0.110907 * mode_Minor


Problem 2 (d)

In [10]:
# index for 14th song
idx = 13  

# feature row (with intercept)
x14 = Phi.iloc[idx].values.reshape(1, -1)   
y_true = float(y.iloc[idx])

# gradient-descent model
pred_gd = float(np.dot(x14, w_gd))    

# closed-form model
pred_cf = float(np.dot(x14, w_cf))          

err_gd = abs(pred_gd - y_true)
err_cf = abs(pred_cf - y_true)

# print song info if available
if 'song_info' in globals() and song_info is not None and len(song_info) > idx:
    title = str(song_info.iloc[idx, 0])
    artist = str(song_info.iloc[idx, 1])
    print("Song:", title, "by", artist)

print("True danceability:", round(y_true, 6))
print("Prediction (gd):", round(pred_gd, 6), "| abs error:", round(err_gd, 6))
print("Prediction (closed-form):", round(pred_cf, 6), "| abs error:", round(err_cf, 6))

closer = "gradient descent" if err_gd < err_cf else "closed-form"
print("Closer prediction:", closer)

Song: Dancing Queen by ABBA
True danceability: 0.543
Prediction (gd): 0.593163 | abs error: 0.050163
Prediction (closed-form): 0.593998 | abs error: 0.050998
Closer prediction: gradient descent


/var/folders/vw/c11105x93rdfqx32_1qmzfn40000gn/T/ipykernel_66153/4176513938.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_gd = float(np.dot(x14, w_gd))
/var/folders/vw/c11105x93rdfqx32_1qmzfn40000gn/T/ipykernel_66153/4176513938.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_cf = float(np.dot(x14, w_cf))


Problem 2 (e)

In [11]:
# Final total error for each method (and R^2)

X = Phi.values.astype(float)
y_vec = y.values.reshape(-1, 1).astype(float)

yhat_gd = np.dot(X, w_gd)
yhat_cf = np.dot(X, w_cf)

sse_gd = float(np.sum((yhat_gd - y_vec) ** 2))
sse_cf = float(np.sum((yhat_cf - y_vec) ** 2))

mse_gd = float(np.mean((yhat_gd - y_vec) ** 2))
mse_cf = float(np.mean((yhat_cf - y_vec) ** 2))

ss_tot = float(np.sum((y_vec - np.mean(y_vec)) ** 2))
r2_gd = 1.0 - sse_gd / ss_tot
r2_cf = 1.0 - sse_cf / ss_tot

print("Total SSE (gd):", round(sse_gd, 6), " | MSE (gd):", round(mse_gd, 6), " | R^2 (gd):", round(r2_gd, 6))
print("Total SSE (closed-form):", round(sse_cf, 6), " | MSE (closed-form):", round(mse_cf, 6), " | R^2 (closed-form):", round(r2_cf, 6))

Total SSE (gd): 0.421923  | MSE (gd): 0.018344  | R^2 (gd): 0.431467
Total SSE (closed-form): 0.421916  | MSE (closed-form): 0.018344  | R^2 (closed-form): 0.431476


Problem 3 Part (a)

In [12]:
from sklearn.preprocessing import PolynomialFeatures

# read file again
spotify = pd.read_csv("ds4400_spotify.csv")

# drop identifiers
spotify = spotify.drop(columns=["song_title", "artist_name"], errors="ignore")

# get the arget
y = spotify["danceability"]

# columns: [1, energy, energy^2]
E = PolynomialFeatures(degree=2, include_bias=True).fit_transform(spotify[["energy"]].values)

# mode one-hot (0/1), becomes the 4th column
mode_minor = pd.get_dummies(spotify[["mode"]], drop_first=True, dtype=int).values


# final feature map Phi2
Phi2 = pd.DataFrame(
    np.column_stack([E, mode_minor]),
    columns=["intercept", "energy", "energy_sq", "mode_Minor"]
)

Phi2["mode_Minor"] = Phi2["mode_Minor"].astype(int)

display(Phi2.head())

,intercept,energy,energy_sq,mode_Minor
0,1.0,0.607,0.368449,0
1,1.0,0.603,0.363609,1
2,1.0,0.764,0.583696,1
3,1.0,0.624,0.389376,0
4,1.0,0.602,0.362404,0


Problem 3 Part (b)

In [13]:
# Polynomial regression with gradient descent using Phi2

X = Phi2.values.astype(float)                   
y_vec = y.values.reshape(-1, 1).astype(float)   
n, p = X.shape

def mse_and_grad(w):
    r = np.dot(X, w) - y_vec
    loss = np.mean(r**2)
    grad = (2.0/n) * np.dot(X.T, r)
    return loss, grad

w = np.zeros((p, 1))
lr = 0.05
max_iter = 10000
tol = 1e-8

loss_hist = []
for i in range(max_iter):
    loss, g = mse_and_grad(w)
    loss_hist.append(loss)
    w -= lr * g
    if i > 0 and abs(loss_hist[-2] - loss_hist[-1]) < tol:
        break

w_gd_poly = w

print("Converged in", i + 1, "iterations")
print("Final MSE:", round(float(loss_hist[-1]), 6))

print("final weights (gd):")
for k, col in enumerate(Phi2.columns):
    print(col, "=", round(float(w_gd_poly[k, 0]), 6))

eq_gd = "f(x) = " + str(round(float(w_gd_poly[0, 0]), 6))
for k, col in enumerate(Phi2.columns[1:], start=1):
    eq_gd += " + " + str(round(float(w_gd_poly[k, 0]), 6)) + " * " + col
print(eq_gd)

Converged in 1053 iterations
Final MSE: 0.028859
final weights (gd):
intercept = 0.519074
energy = 0.100327
energy_sq = -0.050025
mode_Minor = 0.120003
f(x) = 0.519074 + 0.100327 * energy + -0.050025 * energy_sq + 0.120003 * mode_Minor


Problem 3 Part (c)

In [14]:
# Polynomial regression closed-form solution (using Phi2)

X = Phi2.values.astype(float)
y_vec = y.values.reshape(-1, 1).astype(float)

w_cf_poly = np.dot(np.linalg.pinv(X), y_vec)

final_mse_cf = float(np.mean((np.dot(X, w_cf_poly) - y_vec) ** 2))
print("Final MSE (closed-form):", round(final_mse_cf, 6))

print("final weights (closed-form):")
for k, col in enumerate(Phi2.columns):
    print(col, "=", round(float(w_cf_poly[k, 0]), 6))

eq_cf = "f(x) = " + str(round(float(w_cf_poly[0, 0]), 6))
for k, col in enumerate(Phi2.columns[1:], start=1):
    eq_cf += " + " + str(round(float(w_cf_poly[k, 0]), 6)) + " * " + col
print(eq_cf)

Final MSE (closed-form): 0.028854
final weights (closed-form):
intercept = 0.545744
energy = 0.014839
energy_sq = 0.014261
mode_Minor = 0.120724
f(x) = 0.545744 + 0.014839 * energy + 0.014261 * energy_sq + 0.120724 * mode_Minor


Problem 3 Part (d)

In [15]:
# Total error and R^2 for GD vs Closed-Form on Phi2

X2 = Phi2.values.astype(float)
y2 = y.values.reshape(-1, 1).astype(float)

yhat_gd_poly = np.dot(X2, w_gd_poly)
yhat_cf_poly = np.dot(X2, w_cf_poly)

sse_gd_poly = float(np.sum((yhat_gd_poly - y2) ** 2))
sse_cf_poly = float(np.sum((yhat_cf_poly - y2) ** 2))

mse_gd_poly = sse_gd_poly / len(y2)
mse_cf_poly = sse_cf_poly / len(y2)

ss_tot = float(np.sum((y2 - np.mean(y2)) ** 2))
r2_gd_poly = 1.0 - sse_gd_poly / ss_tot
r2_cf_poly = 1.0 - sse_cf_poly / ss_tot

print("Polynomial model (Phi2) — total errors and R^2")
print("SSE (gd):", round(sse_gd_poly, 6), " MSE (gd):", round(mse_gd_poly, 6), " R^2 (gd):", round(r2_gd_poly, 6))
print("SSE (cf):", round(sse_cf_poly, 6), " MSE (cf):", round(mse_cf_poly, 6), " R^2 (cf):", round(r2_cf_poly, 6))

Polynomial model (Phi2) — total errors and R^2
SSE (gd): 0.663761  MSE (gd): 0.028859  R^2 (gd): 0.105596
SSE (cf): 0.663634  MSE (cf): 0.028854  R^2 (cf): 0.105767


Problem 3 Part (e)

In [16]:
# Pick the best overall model, Problem 2 vs Problem 3

# metrics for Problem 2 (linear model from part (a): Phi, w_gd, w_cf)
X = Phi.values.astype(float)
y_vec = y.values.reshape(-1, 1).astype(float)

yhat_gd_lin = np.dot(X, w_gd)
yhat_cf_lin = np.dot(X, w_cf)

sse_gd_lin = float(np.sum((yhat_gd_lin - y_vec) ** 2))
sse_cf_lin = float(np.sum((yhat_cf_lin - y_vec) ** 2))
mse_gd_lin = sse_gd_lin / len(y_vec)
mse_cf_lin = sse_cf_lin / len(y_vec)
ss_tot = float(np.sum((y_vec - np.mean(y_vec)) ** 2))
r2_gd_lin = 1.0 - sse_gd_lin / ss_tot
r2_cf_lin = 1.0 - sse_cf_lin / ss_tot

# Summary: mse_gd_poly, mse_cf_poly, r2_gd_poly, r2_cf_poly
candidates = [
    ("P2-GD  (linear)", r2_gd_lin, mse_gd_lin),
    ("P2-CF  (linear)", r2_cf_lin, mse_cf_lin),
    ("P3-GD  (poly2)",  r2_gd_poly, mse_gd_poly),
    ("P3-CF  (poly2)",  r2_cf_poly, mse_cf_poly),
]
best = sorted(candidates, key=lambda t: (t[1], -t[2]), reverse=True)[0]

print("Summary (R^2, MSE)")
print("P2-GD :", round(r2_gd_lin, 6), round(mse_gd_lin, 6))
print("P2-CF :", round(r2_cf_lin, 6), round(mse_cf_lin, 6))
print("P3-GD :", round(r2_gd_poly, 6), round(mse_gd_poly, 6))
print("P3-CF :", round(r2_cf_poly, 6), round(mse_cf_poly, 6))

Summary (R^2, MSE)
P2-GD : 0.431467 0.018344
P2-CF : 0.431476 0.018344
P3-GD : 0.105596 0.028859
P3-CF : 0.105767 0.028854


The best model is the closed form solution from problem 2. It has a higher R^2 and lower MSE, indicating that it has a better fit on this dataset. I think some improvements could be to ddd more features (valence, loudness^2, tempo×mode), use regularization (ridge or lasso) to reduce variance and stabilize weights, and lastly, a train/validation split or cross-validation to check generalization. We could then tune the degree and the learning rate.